# সিফ্ট কেরাসের সাথে ব্যক্তিগত ভবিষ্যদ্বাণী (Private Predictions with Syft Keras)

## পদক্ষেপ 3: সিফ্ট কেরাস ব্যবহার করে ব্যক্তিগত ভবিষ্যদ্বাণী - পরিবেশন করা (ক্লায়েন্ট)  (Private Prediction using Syft Keras - Serving (Client)

অভিনন্দন! আপনার মডেলটিকে সাধারণ কেরাস দিয়ে প্রশিক্ষণ দেওয়ার পরে এবং এটি সিফ্ট কেরাস দিয়ে সুরক্ষিত করার পরে, আপনি কিছু ব্যক্তিগত ভবিষ্যদ্বাণীগুলির অনুরোধ করতে প্রস্তুত।

অনুবাদক:

- Sourav Das - Twitter: [@adventuroussrv](https://twitter.com/adventuroussrv)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import syft as sy

## ডেটা (Data)

এখানে, আমরা আমাদের এমএনআইএসটি ডেটা(MNIST data) প্রাক প্রক্রিয়াজাত(preprocessed) করি। এটি আমরা প্রশিক্ষণের সময় যেভাবে প্রাক-প্রক্রিয়াজাতকরণের সেই অনুরূপ।

In [2]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## মডেলটি সংযুক্ত করুন (Connect to model)

মডেলটি জিজ্ঞাসাবাদ করার আগে, আপনাকে এটির সাথে সংযোগ করতে হবে। এটি করতে, আপনি একটি ক্লায়েন্ট তৈরি করতে পারেন। তারপর সঠিক একই তিন TFEWorkers (alice, bob এবং carol) এবং ক্লাস্টার। অবশেষে কল করুন connect_to_model। এটি ক্লায়েন্ট সাইডে একটি TFE জিজ্ঞাসাবাদ সার্ভার তৈরি করে model.serve() যা Part 13b দ্বারা স্থাপন করা কুইউং সার্ভারের সাথে সংযোগ স্থাপন করে । পূর্বাভাসের অনুরোধে শেয়ারগুলি জমা দেওয়ার আগে সহজ লেখা ডেটা গোপনে ভাগ করে নেওয়ার জন্য সারিটি দায়বদ্ধ থাকবে।

In [3]:
num_classes = 10
input_shape = (1, 28, 28, 1)
output_shape = (1, num_classes)

In [4]:
client = sy.TFEWorker()

alice = sy.TFEWorker(host='localhost:4000')
bob = sy.TFEWorker(host='localhost:4001')
carol = sy.TFEWorker(host='localhost:4002')
cluster = sy.TFECluster(alice, bob, carol)

client.connect_to_model(input_shape, output_shape, cluster)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



## অনুসন্ধান মডেল (Query model)

আপনি কিছু ব্যক্তিগত ভবিষ্যদ্বাণী পেতে প্রস্তুত! query_model কল করা চিত্রটিকে উপরের তৈরি কাতারে সন্নিবেশ করবে, গোপনীয়ভাবে স্থানীয়ভাবে ডেটা ভাগ করবে এবং Part 13b তে মডেল সার্ভারে শেয়ার জমা দেবে।

In [5]:
# User inputs
num_tests = 3
images, expected_labels = x_test[:num_tests], y_test[:num_tests]

In [6]:
for image, expected_label in zip(images, expected_labels):

    res = client.query_model(image.reshape(1, 28, 28, 1))
    predicted_label = np.argmax(res)

    print("The image had label {} and was {} classified as {}".format(
        expected_label,
        "correctly" if expected_label == predicted_label else "wrongly",
        predicted_label))

The image had label 7 and was correctly classified as 7
The image had label 2 and was correctly classified as 2
The image had label 1 and was correctly classified as 1


এটা অসাধারণ. আপনি এই তিনটি চিত্রকে সঠিকভাবে শ্রেণিবদ্ধ করতে সক্ষম! তবে এই পূর্বাভাসগুলির মধ্যে বিশেষটি হ'ল আপনি এই পরিষেবাটি পাওয়ার জন্য কোনও ব্যক্তিগত তথ্য প্রকাশ করেন নি। মডেল হোস্ট আপনার ইনপুট ডেটা বা আপনার পূর্বাভাস কখনও দেখেনি এবং আপনি কখনও মডেলটি ডাউনলোড করেন নি। আপনি একটি এনক্রিপ্ট করা মডেল সহ এনক্রিপ্ট করা ডেটাতে ব্যক্তিগত ভবিষ্যদ্বাণী পেতে সক্ষম হয়েছিলেন!

এটি আমাদের নিজস্ব অ্যাপ্লিকেশনগুলিতে প্রয়োগ করার জন্য ছুটে যাওয়ার আগে, আমাদের পরিবেশন করা মডেলটি পরিষ্কার করতে দ্রুত  Part 13b তে ফিরে যাই!